In [ ]:
# import the dependencies
import pandas as pd
import ast

In [ ]:
df = pd.read_csv("./listings__20251027_061613.csv")

In [ ]:
df_unclean = df.copy()

In [60]:
df_unclean.head()

,url,title,location,house_type,bathrooms,bedrooms,properties,amenities,price
0,https://jiji.com.gh/north-legon/houses-apartme...,1bdrm Apartment in North Legon for rent,"Greater Accra, North Legon, 3 hours ago",Apartment,1 bathroom,1 bedroom,"{'Property Address': 'North Legon', 'Property ...",NaN,"GH₵ 2,500"
1,https://jiji.com.gh/school-junction/houses-apa...,"2bdrm Apartment in Nanakrom, School Junction f...","Greater Accra, Ashaley Botwe, Ashaley Botwe Sc...",Apartment,2 bathrooms,2 bedrooms,"{'Property Address': 'School Junction', 'Estat...",NaN,"GH₵ 3,000"
2,https://jiji.com.gh/malejor/houses-apartments-...,"2bdrm Apartment in Adenta Dodowa, Malejor for ...","Greater Accra, Adenta, Malejor, 3 hours ago",Apartment,2 bathrooms,2 bedrooms,"{'Property Address': 'Major', 'Estate Name': '...",NaN,"GH₵ 2,000"
3,https://jiji.com.gh/cantonments/houses-apartme...,"4bdrm House in Cantonment, Cantonments for rent","Greater Accra, Cantonments, 3 hours ago",House,4 bathrooms,4 bedrooms,"{'Property Address': 'Cantonment', 'Estate Nam...",NaN,"GH₵ 72,000"
4,https://jiji.com.gh/american-house/houses-apar...,1bdrm Apartment in American House for rent,"Greater Accra, East Legon, American House, 3 h...",Apartment,1 bathroom,1 bedroom,"{'Property Address': 'American House', 'Proper...",NaN,"GH₵ 3,500"


In [ ]:
# grab all columns
df_unclean.columns

Index(['url', 'title', 'location', 'house_type', 'bathrooms', 'bedrooms',
       'properties', 'amenities', 'price'],
      dtype='object')

In [62]:
df_unclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16356 entries, 0 to 16355
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   url         16356 non-null  object
 1   title       16356 non-null  object
 2   location    16356 non-null  object
 3   house_type  16327 non-null  object
 4   bathrooms   16353 non-null  object
 5   bedrooms    16353 non-null  object
 6   properties  16356 non-null  object
 7   amenities   13991 non-null  object
 8   price       16356 non-null  object
dtypes: object(9)
memory usage: 1.1+ MB


In [ ]:
def getSubLoc(loc):
    parts = str(loc).split(",")
    if len(parts) >= 3:
        return parts[1].strip()
    return parts[0]


df["sub_loc"] = df["location"].apply(getSubLoc)

In [ ]:
def clean_all(data):
    # change the dictionary-like string  into a real dictionary
    data["prop_dict"] = data["properties"].apply(ast.literal_eval)

    # Extracting some variables

    # data["new_location"] = data["location"].str.split(",", expand=True)[1]
    data["distance1"] = data["location"].str.split(",", expand=True)[2]
    data["distance2"] = data["location"].str.split(",", expand=True)[3]
    data["n_bathrooms"] = data["bathrooms"].str.split(" ", expand=True)[0]
    data["n_bedroomns"] = data["bedrooms"].str.split(" ", expand=True)[0]
    data["Prices"] = (
        data["price"].str.split(" ", expand=True)[1].str.replace(",", "").astype(float)
    )

    # drop some redundant variabbles
    data.drop(
        ["url", "title", "location", "bathrooms", "bedrooms", "properties", "price"],
        axis=1,
        inplace=True,
    )


clean_all(df_unclean)

In [ ]:
def extract_prop(property_info):
    if isinstance(property_info, dict):
        return pd.Series(
            {
                "Property Address": property_info.get("Property Address", None),
                "Property Size": property_info.get("Property Size", None),
                "Condition": property_info.get("Condition", None),
                "Furnishing": property_info.get("Furnishing", None),
                "Minimum Rental Period": property_info.get(
                    "Minimum Rental Period", None
                ),
            }
        )
    return pd.Series(
        {
            "Property Address": None,
            "Property Size": None,
            "Condition": None,
            "Furnishing": None,
            "Minimum Rental Period": None,
        }
    )


df_unclean[
    [
        "Property Address",
        "Property Size",
        "Condition",
        "Furnishing",
        "Minimum Rental Period",
    ]
] = df_unclean["prop_dict"].apply(extract_prop)

In [ ]:
df_unclean.drop("prop_dict", axis=1, inplace=True)

In [66]:
df_unclean.head()

,house_type,amenities,new_location,distance1,distance2,n_bathrooms,n_bedroomns,Prices,Property Address,Property Size,Condition,Furnishing,Minimum Rental Period
0,Apartment,NaN,North Legon,3 hours ago,None,1,1,2500.0,North Legon,15,Fairly Used,Unfurnished,12 Month
1,Apartment,NaN,Ashaley Botwe,Ashaley Botwe School Junction,3 hours ago,2,2,3000.0,School Junction,25,Newly-Built,Unfurnished,12 Month
2,Apartment,NaN,Adenta,Malejor,3 hours ago,2,2,2000.0,Major,100,Newly-Built,Semi-Furnished,12 Month
3,House,NaN,Cantonments,3 hours ago,None,4,4,72000.0,Cantonment,5000,Newly-Built,Unfurnished,None
4,Apartment,NaN,East Legon,American House,3 hours ago,1,1,3500.0,American House,15,Fairly Used,Unfurnished,12 Month


In [ ]:
df_unclean["new_location"].unique()

array([' North Legon', ' Ashaley Botwe', ' Adenta', ' Cantonments',
       ' East Legon', ' Teshie', ' Abokobi', ' Oyarifa', ' Achimota',
       ' Ga West Municipal', ' Amrahia', ' Dome', ' Tantra Hills',
       ' Spintex', ' Adjiriganor', ' Abelemkpe', ' Pokuase',
       ' McCarthy Hill', ' Agbogba', ' West Legon', ' Old Ashomang',
       ' Accra Metropolitan', ' Airport Residential Area', ' Haatso',
       ' Tema Metropolitan', ' Taifa-Burkina', ' Gbawe', ' Ablekuma',
       ' Weija', ' Madina', ' Awoshie', ' Nungua', ' Ofankor',
       ' Dansoman', ' Oyibi', ' Ga South Municipal', ' Ashomang Estate',
       ' Ashaiman Municipal', ' Odorkor', ' Lapaz', ' Kasoa', ' Osu',
       ' Sowutwuom', ' Dzorwulu', ' Kaneshie', ' Darkuman', ' Mataheko',
       ' 2 hours ago', ' Kotobabi', ' Lartebiokoshie', ' Tesano',
       ' Labadi', ' Anyaa', ' Labone', ' Ga East Municipal',
       ' Burma Camp', ' Ridge', ' Dworwulu', ' 1 hour ago',
       ' Asylum Down', ' Danfa', ' Santa Maria', ' North In

In [ ]:
df["properties"].iloc[37]

"{'Property Address': 'Oyarifa', 'Estate Name': 'Oyarifa', 'Property Size': '100', 'Subtype': 'Detached', 'Condition': 'Newly-Built', 'Furnishing': 'Unfurnished', 'Toilets': '4'}"